In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import re

In [2]:
df = pd.read_csv(r'df.csv')

C:\Users\SALAH MAHMOUD\AppData\Local\Temp\ipykernel_1704\4052145175.py:1: DtypeWarning: Columns (10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'df.csv')


In [3]:
df.head(2)

,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,Gross Weight,Net Weight,Package Weight
0,"Single Female with four children, education: b...",H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,recyclable,10K+,2842.23,2037.64,481.98,323,602.7575,28.1997,26.6008,1.599
1,"Single Female with three children, education: ...",S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650,16.5710,14.9720,1.599


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38862 entries, 0 to 38861
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Person Description             38862 non-null  object 
 1   Place Code                     38862 non-null  object 
 2   Customer Order                 38860 non-null  object 
 3   Additional Features in market  34562 non-null  object 
 4   Promotion Name                 33034 non-null  object 
 5   Store Kind                     33053 non-null  object 
 6   Store Sales                    38847 non-null  object 
 7   Store Cost                     38855 non-null  object 
 8   Is Recyclable?                 38860 non-null  object 
 9   Yearly Income                  38854 non-null  object 
 10  Store Area                     35297 non-null  object 
 11  Grocery Area                   35256 non-null  object 
 12  Frozen Area                    35310 non-null 

# Let's perform data cleaning and create new features😉

In [5]:
# Apply the 'dropna' function to the target column, and then fix the other columns after exploring the data                        
df = df.dropna(subset=["Cost"], axis=0)

In [6]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [7]:
for col in ['Store Area', 'Grocery Area', 'Frozen Area', 'Meat Area']:
    # Convert to numeric (float) and handle missing values
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].replace('missing', float('nan'))

In [8]:
# Removing 'Millions','illions','M' and converting 
for col in ['Store Sales', 'Store Cost']:
    df[col] = df[col].astype(str).str.replace('Millions', '').str.replace('illions', '').str.replace('M', '').astype(float) * 1000000

In [9]:
# Function to extract city&country codes
def extract_city_code(place_code):
    parts = place_code.split('_')  # Split by underscore
    if len(parts) == 2:
        return parts[0]
    else:
        return None
# Function to extract city&country codes
def extract_country_code(place_code):
    parts = place_code.split('_')  # Split by underscore
    if len(parts) == 2:
        return parts[1]
    else:
        return None

# Apply the function to create a new 'City Code' column
df['City Code'] = df['Place Code'].apply(extract_city_code)
df['Country Code'] = df['Place Code'].apply(extract_country_code)



In [10]:
df.drop('Place Code',axis=1,inplace = True)

In [11]:
# Extracts marital status (Single or Married) from text
def extract_status(text):
    status_pattern = r'Single|Married'  # Match Single or Married
    status_match = re.search(status_pattern, text)
    return status_match.group() if status_match else None

# Extracts the number of children from text
def extract_children(text):
    children_pattern = r'\b(\w+) children\b'  # Match [word] children
    children_match = re.search(children_pattern, text)
    return children_match.group(1) if children_match else None

# Extracts education information from text
def extract_education(text):
    education_pattern = r'education:\s*(\w+\s*\w+)'  # Match education: [word word]
    education_match = re.search(education_pattern, text, re.IGNORECASE)
    return education_match.group(1) if education_match else None

# Extracts working status from text
def extract_working(text):
    working_pattern = r'as (\w+(?: \w+)*)'  # Match as [word(s)]
    working_match = re.search(working_pattern, text, re.IGNORECASE)
    return working_match.group(1) if working_match else None

# Extracts gender (Male or Female) from text
def extract_sex(text):
    sex_pattern = r'\b(?:Male|Female)\b'  # Match Male or Female
    sex_match = re.search(sex_pattern, text, re.IGNORECASE)
    return sex_match.group() if sex_match else None

# Loop through each DataFrame column
df['Status'] = df['Person Description'].apply(extract_status)
df['Number of Children'] = df['Person Description'].apply(extract_children)
df['Working Status'] = df['Person Description'].apply(extract_working)
df['Sex'] = df['Person Description'].apply(extract_sex)

In [12]:
df.drop('Person Description',axis=1,inplace = True)

In [13]:
# Extracts product type from text
def extract_product_type(text):
    pattern = r'^([\w\s]+) from' 
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

# Extracts ordered brand from text
def extract_ordered_brand(text):
    pattern = r'Ordered Brand : ([\w\s]+)' 
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

# Extracts department from text
def extract_department(text):
    pattern = r'from ([\w\s]+) department'
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

# Loop through each DataFrame column
df['Department'] = df['Customer Order'].apply(extract_department) 
df['Ordered_Brand'] = df['Customer Order'].apply(extract_ordered_brand)  
df['Product_Type'] = df['Customer Order'].apply(extract_product_type)  


In [14]:
df.drop('Customer Order',axis=1,inplace = True)

In [15]:
# make new columns from Additional Features in market
def extract_feature(text, feature_name):
    if pd.isna(text):
        return np.nan
    elif feature_name in text:
        return 1
    else:
        return 0

# Define a list of features to extract
features_to_extract = ['Video Store', 'Florist', 'Coffee Bar', 'Bar For Salad', 'Ready Food']

# Loop through the list of features and apply the extraction function
for feature in features_to_extract:
    df[feature] = df['Additional Features in market'].apply(lambda x: extract_feature(x, feature))

In [16]:
df.drop('Additional Features in market',axis=1,inplace = True)

In [17]:
# Mapping of textual values to numerical values
mapping = {
    'No': 0,
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    # Add more mappings as needed
}

# Replace textual values with numerical values
df['Number of Children'] = df['Number of Children'].replace(mapping)

# Convert the column to numeric (this will handle NaN values as well)
df['Number of Children'] = pd.to_numeric(df['Number of Children'], errors='coerce')


In [18]:
# clean Yearly Income
df['Yearly Income'] = df['Yearly Income'].str.replace("K", "").str.replace("+", "").str.replace(".", "").astype('float64')

C:\Users\SALAH MAHMOUD\AppData\Local\Temp\ipykernel_1704\2051523988.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Yearly Income'] = df['Yearly Income'].str.replace("K", "").str.replace("+", "").str.replace(".", "").astype('float64')


In [19]:
# Define a function 'Income' that categorizes income values
def Income(x):
    if x <= 70:
        return 'less_70'  # If income is less than or equal to 70, categorize as 0
    elif x > 70:
        return 'greater_70'  # If income is greater than 70, categorize as 1

df['Yearly Income'] = df['Yearly Income'].apply(Income)

In [20]:
def Is_Recyclable(x):
    if x == 'recyclable':
        return 'yes'
    elif x == 'non recyclable':  
        return 'no'
    elif x == 'yes':
        return 'yes'
    elif x == 'no':return 'no'

df['Is Recyclable?'] = df['Is Recyclable?'].apply(Is_Recyclable)

In [21]:
pd.set_option('display.max_columns', 28)
df.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,Gross Weight,Net Weight,Package Weight,City Code,Country Code,Status,Number of Children,Working Status,Sex,Department,Ordered_Brand,Product_Type,Video Store,Florist,Coffee Bar,Bar For Salad,Ready Food
0,Dimes Off,Deluxe,8760000.0,4292400.0,yes,less_70,2842.23,2037.64,481.98,323.00,602.7575,28.1997,26.6008,1.5990,H11go,ZA,Single,4.0,professional,Female,Household,Red Wing,Cleaning Supplies,1.0,1.0,1.0,1.0,1.0
1,Budget Bargains,Supermarket,6360000.0,1971600.0,no,less_70,2814.95,2049.72,457.36,NaN,708.6650,16.5710,14.9720,1.5990,S04ne,WA,Single,3.0,management,Female,Snack Foods,Nationeel,Snack Foods,NaN,NaN,NaN,NaN,NaN
2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,yes,less_70,2192.32,1322.21,523.32,348.85,564.2647,28.6358,27.1822,1.4536,L05es,CA,Married,2.0,skilled manual,Male,Periodicals,Excel,Magazines,0.0,1.0,0.0,0.0,0.0
3,Savings Galore,NaN,1980000.0,673200.0,yes,less_70,1974.73,NaN,440.92,293.95,724.5119,14.2161,11.2944,2.9217,S03le,WA,Single,3.0,professional,Female,Dairy,Carlson,Dairy,0.0,0.0,1.0,0.0,0.0
4,Sale Winners,Deluxe,11560000.0,4970800.0,no,less_70,2862.30,1872.19,593.93,395.95,519.7574,12.6172,9.7100,2.9072,M10da,YU,Married,5.0,professional,Female,Produce,Hermanos,Vegetables,1.0,1.0,1.0,1.0,1.0


In [22]:
df.to_csv("Preprocessing.csv", index=False)

# Let's explore the data.😊